In [45]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

url = 'https://www.lazada.co.th/catalog/?_keyori=ss&from=search_history&page={1}&q={0}&spm=a2o4m.home.search.1.1125515ftmsdov&sugg={0}0_1'
#{0}:format後的第一個位置、{1}:第二個


def get_urls(url, query, start_page, end_page): #建立可自行輸入"查詢值"和"start~end_page"的url清單
    urls = []
    for page in range(start_page, end_page+1):
        urls.append(url.format(query, page))    #query帶入url的{0}、page帶入{1}
    return urls

def get_parse(url):
    headers = {"user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Mobile Safari/537.36"}
    res = requests.get(url, headers=headers)
    res = requests.get(url)
    if res.status_code == requests.codes.ok:
        print("Request Succeeded")
        return BeautifulSoup(res.text, "lxml") #常用的解析器有三種："lxml","html5lib","html.parser"，官方建議使用解析速度較快的"lxml"
    else:
        return print("Request Failed")
    
def get_goods(soup):
    goods = []
    rows = soup.find("div", attrs={"class":"_1HmYoV _35HD7C", "style":"flex-grow:1;overflow:auto"}).find_all("div", "_3liAhj")
    #rows = soup.selct("._3liAhj")
    for row in rows:
        try:
            name = row.find("a", attrs={"class":"_2cLu-l"}).text
            #name = row.select("._2cLu-l")[0].get_text()   #後面加[0]代表只找第一個，等同為select_one
            #name = row.find("div", class_="_3BTv9X").img["alt"]
        except:
            name = None
        try:
            #price = row.find("div", attrs={"class":"_1vC4OE"}).text
            price = row.select_one("._1vC4OE").get_text()
        except:
            price = None
        try:
            #price_2 = row.find("div", "_3auQ3N").text
            price_2 = row.select("._3auQ3N")[0].get_text()
        except:
            price_2 = None
        try:
            #discount = row.find("div", "VGWI6T").text
            discount = row.select_one(".VGWI6T span").get_text()
        except:
            discount = None
        try:
            #star = row.find("div", "hGSR34").text
            star = row.select(".hGSR34")[0].get_text()
        except:
            star = None
        try:
            #reviews = row.find("span", "_38sUEc").text
            reviews = row.select("._38sUEc")[0].get_text()
        except:
            reviews = None
            
        good = [name, price, price_2, discount, star, reviews]  #將爬取項目照此順序放入清單
        goods.append(good)  #將good清單加入goods清單內，會產生[[name, price, price_2, discount, star, reviews]]巢狀清單
    return goods

def scraping_bot(urls):
    all_goods = [["品名", "價格", "價格2", "折扣%", "評價", "評論數"]]  #巢狀清單(當作欄位名稱用)
    
    page = 1
    for url in urls:
        print("Scraping_Page" + str(page) + "...")
        page = page + 1
        soup = get_parse(url)
        goods = get_goods(soup)
        all_goods = all_goods + goods
        print("Wait 3 Seconds...")
        if soup.find("li", class_="a-disabled a-last"):  #每個網頁都不同，要自行改，但如果有限制end_page的話不寫這段也沒差
            break   #已經沒有下一頁
        time.sleep(3)
    return all_goods


def save_to_csv(items, file):
    with open(file, "w+", newline="", encoding="utf_8_sig") as fp:  #utf_8_sig:能讓輸出的csv正確顯示中文(utf_8會有亂碼)
        writer = csv.writer(fp)
        for item in items:
            writer.writerow(item)


#爬蟲主程式
if __name__ == "__main__":  #當被匯入時不執行的段落
#get_urls
    urls = get_urls(url, 'logitech', 1, 1)
    print(urls)
    '''
#get_data
    goods = scraping_bot(urls)
    df = pd.DataFrame(goods)     #用dataframe列出
    print(df)
    #for good in goods:          #用list列出
    #    print(good)
    save_to_csv(goods, "t.csv")
    '''

['https://www.lazada.co.th/catalog/?_keyori=ss&from=search_history&page=1&q=logitech&spm=a2o4m.home.search.1.1125515ftmsdov&sugg=logitech0_1']


In [67]:
get_parse("https://shopee.com.my/api/v2/search_items/?by=relevancy&keyword=genius%20mouse&limit=50&newest=0&order=desc&page_type=search&version=2")

Request Succeeded


<html><body><p>{"show_disclaimer":null,"query_rewrite":null,"adjust":null,"version":"b1c94828d525e526ff969f451cc3ac33","algorithm":null,"total_count":null,"error":null,"total_ads_count":null,"nomore":null,"price_adjust":null,"json_data":null,"suggestion_algorithm":null,"items":null,"reserved_keyword":null,"hint_keywords":null}</p></body></html>

In [68]:
get_parse("https://shopee.com.my/search?keyword=genius%20mouse")


Request Succeeded


<!DOCTYPE html>
<html>
<head>
<link href="//cf.shopee.com.my/" rel="preconnect"/>
<link href="//deo.shopeemobile.com/shopee/" rel="preconnect"/>
<link href="//cv.shopee.com.my/" rel="preconnect"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1, minimum-scale=1, user-scalable=no, viewport-fit=cover" name="viewport"/>
<meta content="3a3f2fed3bf2bb06ff212e02a5aa593609e3b68c" name="shopee:git-sha"/>
<meta content="rw-v3.34.0" name="shopee:version"/>
<script>
      window.__LOCALE__ = 'MY';
      window.__ENV__ = 'live';
      window.__ASSETS__ = {"TRANSIFY_MAPPING":{"id-live":1583496273,"hi-live":1583480054,"ms-my-live":1583479984,"pt-br-live":1583480043,"vi-live":1583479970,"en-live":1583482453,"zh-hans-live":1583480011,"zh-hant-live":1583999066,"th-live":1583480022},"CCMS_FEATURE_TOGGLE":{"ALL_CATEGORIES_HOMEPAGE_ENTRANCE":{"default":false,"TW":true,"VN":true,"PH":true,"MY":true,"ID":true},"AMP_CATEGORY_PAGE":{"default":true},"BUNDLE_DEALS":{"

In [39]:
df

,0,1,2,3,4,5
0,品名,價格,價格2,折扣%,評價,評論數
1,Logitech M100R Wired Optical Mouse,₹449,₹495,9% off,3.9,"(1,662)"
2,Logitech M90 Wired Optical Gaming Mouse,₹349,₹395,11% off,4.3,"(1,042)"
3,Logitech Mk235 Mouse & Wireless Laptop Keyboard,"₹1,299","₹1,995",34% off,4.3,"(1,675)"
4,Logitech MK240 Wireless Keyboard and Mouse Combo,"₹1,399","₹1,500",6% off,4.4,"(23,710)"
...,...,...,...,...,...,...
196,Logitech Dotionmo Mousepad,₹125,₹150,16% off,None,None
197,Logitech H570e Wired Headset,"₹4,900","₹4,995",1% off,None,None
198,Hermitshell Keyboard Case for Logitech,"₹3,205","₹4,167",23% off,None,None
199,"Logitech HD Webcam C525, Portable HD 720p Vide...","₹3,550","₹4,295",17% off,4.1,(14)


In [3]:
ds = []
d = ['name', 'price', 'price_2', 'discount', 'star, reviews']
ds.append(d)

In [4]:
ds

[['name', 'price', 'price_2', 'discount', 'star, reviews']]

In [63]:
from selenium import webdriver
driver = webdriver.Chrome()
driver.get('https://shopee.com.my/search?keyword=genius%20mouse')

In [ ]:
#使用selenium自行產生cookies
c = ';'.join(['{}={}'.format(item.get('name'), item.get('value')) for item in driver.get_cookies()])

In [ ]:
#使用selenium自行產生x-csrftoken
token = [item.get('value') for item in driver.get_cookies() if item.get('name') == 'csrftoken'][0]

In [ ]:
import requests
import json
jd = json.loads('{"item_shop_ids":[{"itemid":30348675,"adsid":0,"shopid":2101724,"campaignid":0},{"itemid":30193672,"adsid":0,"shopid":4067127,"campaignid":0},{"itemid":9866816,"adsid":0,"shopid":1249908,"campaignid":0},{"itemid":4079412,"adsid":0,"shopid":1556792,"campaignid":0},{"itemid":3326952,"adsid":0,"shopid":196610,"campaignid":0},{"itemid":29678731,"adsid":0,"shopid":4063412,"campaignid":0},{"itemid":29893101,"adsid":0,"shopid":2519614,"campaignid":0},{"itemid":4278917,"adsid":0,"shopid":1979042,"campaignid":0},{"itemid":19967138,"adsid":0,"shopid":482725,"campaignid":0},{"itemid":29027205,"adsid":0,"shopid":3458145,"campaignid":0},{"itemid":7658646,"adsid":0,"shopid":1975307,"campaignid":0},{"itemid":15614334,"adsid":0,"shopid":2896296,"campaignid":0},{"itemid":14475174,"adsid":0,"shopid":118223,"campaignid":0},{"itemid":13837697,"adsid":0,"shopid":3625341,"campaignid":0},{"itemid":3563733,"adsid":0,"shopid":1627706,"campaignid":0},{"itemid":13656164,"adsid":0,"shopid":2157465,"campaignid":0},{"itemid":7450775,"adsid":0,"shopid":2119456,"campaignid":0},{"itemid":14147967,"adsid":0,"shopid":3500523,"campaignid":0},{"itemid":2855240,"adsid":0,"shopid":1241858,"campaignid":0},{"itemid":19785078,"adsid":0,"shopid":5435363,"campaignid":0},{"itemid":26867254,"adsid":0,"shopid":3434330,"campaignid":0},{"itemid":15322004,"adsid":0,"shopid":2618335,"campaignid":0},{"itemid":15001686,"adsid":0,"shopid":4634210,"campaignid":0},{"itemid":29047156,"adsid":0,"shopid":5435363,"campaignid":0},{"itemid":15462067,"adsid":0,"shopid":2366165,"campaignid":0},{"itemid":11472312,"adsid":0,"shopid":159837,"campaignid":0},{"itemid":31053194,"adsid":0,"shopid":5356071,"campaignid":0},{"itemid":14971259,"adsid":0,"shopid":691981,"campaignid":0},{"itemid":13472705,"adsid":0,"shopid":159837,"campaignid":0},{"itemid":12209945,"adsid":0,"shopid":2382253,"campaignid":0},{"itemid":1854831,"adsid":0,"shopid":12443,"campaignid":0},{"itemid":18885271,"adsid":0,"shopid":1118225,"campaignid":0},{"itemid":2609434,"adsid":0,"shopid":1241858,"campaignid":0},{"itemid":9078614,"adsid":0,"shopid":1627706,"campaignid":0},{"itemid":18754381,"adsid":0,"shopid":4668818,"campaignid":0},{"itemid":2329695,"adsid":0,"shopid":196610,"campaignid":0},{"itemid":13547419,"adsid":0,"shopid":118223,"campaignid":0},{"itemid":2500976,"adsid":0,"shopid":196610,"campaignid":0},{"itemid":14344683,"adsid":0,"shopid":1889147,"campaignid":0},{"itemid":31003378,"adsid":0,"shopid":2450531,"campaignid":0},{"itemid":3213746,"adsid":0,"shopid":196610,"campaignid":0},{"itemid":18589831,"adsid":0,"shopid":633256,"campaignid":0},{"itemid":2407439,"adsid":0,"shopid":196610,"campaignid":0},{"itemid":15524401,"adsid":0,"shopid":2276618,"campaignid":0},{"itemid":14483747,"adsid":0,"shopid":118223,"campaignid":0},{"itemid":7501326,"adsid":0,"shopid":2118746,"campaignid":0},{"itemid":18485669,"adsid":0,"shopid":118223,"campaignid":0},{"itemid":4852403,"adsid":0,"shopid":1982863,"campaignid":0},{"itemid":15984364,"adsid":0,"shopid":4160426,"campaignid":0},{"itemid":2885334,"adsid":0,"shopid":118223,"campaignid":0}]}')
headers = {
'Referer':'https://shopee.tw/%E6%89%8B%E6%A9%9F%E5%B9%B3%E6%9D%BF%E8%88%87%E5%91%A8%E9%82%8A-cat.70',
'Cookie': c,
'x-csrftoken':token,
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36'    
}
res = requests.post('https://shopee.tw/api/v1/items/', json = jd, headers = headers)
res

In [ ]:
import pandas
df = pandas.DataFrame(res.json())
df